<a href="https://colab.research.google.com/github/eusebiogerman/WEBSCRAPING/blob/master/nba_webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import unittest
import time
import sys
import os
from os import path
import traceback
import requests
import logging

from console_progressbar import ProgressBar
from bs4 import BeautifulSoup , Comment 
from lxml import etree

#test
def newtag(bs : BeautifulSoup, tagn : str ,text: str) :
    tag = bs.new_tag(tagn)
    tag.string = text 
    return tag

def get_trace_value(plist):
    trace = ''
    for p in plist:
        trace = trace + p + '/n'
    return trace

def load_nba_schedule(year):
    
    #BASE_URL = "https://www.baseball-reference.com" 
    BASE_URL = "https://www.basketball-reference.com/"
    link = f"{BASE_URL}/leagues/NBA_{year}_games.html"
    link2 = ''
    archivo = ''
    fecha = ''
    schedule = []
    gdir = str(year)
    dummycnt = 1
    
    try:
        #crear el repots
        if not (path.isdir('sessons')):
            os.mkdir('sessons')

        #carga el archivo de los datos
        #file = open("mlb_game.txt","a")

        # cargar la pagina
        time.sleep(3)
        tic = time.perf_counter()
        reps = requests.get(link)
        toc = time.perf_counter()
        
        #leer el contenido de la pagina
        html = reps.content
        if (html):  
            print(f"Schedule {toc - tic+1:0.4} seconds url: {link}")
            soup = BeautifulSoup(html,'html.parser')
            div_schedule = soup.find_all('div', {'id','div_schedule'})
            table = soup.find_all('table')
            tb = table[0].find_all('tbody')
            trs = tb[0].find_all('tr')

            for tr in trs:
                tds = tr.find_all('td', {'class','center'})
                for td in tds:
                    a = td.find_all('a')
                    if (a) :
                       dict = {}
                       fecha = a[0].get('href').split('/')[2][3:12]
                       archivo = f"sessons/{gdir}.txt"
                       link2 =  f"{BASE_URL}{a[0].get('href')}"
                       dict ={'link' : link2, 'archivo' : archivo, 'fecha' : fecha }  
                       schedule.append(dict)
        return schedule
    except ValueError:
            print("Error de Valor")
            traceback.print_exception(*sys.exc_info())
            return schedule
    except:    
            print("Error de general") 
            traceback.print_exception(*sys.exc_info())   
            return schedule                     



sche = load_nba_schedule(2018) 
for es in sche:
    print(es)
#score  = 'https://www.baseball-reference.com/boxes/CHN/CHN201810020.shtml'                                            
#load_mlb_games(score,2018)


# https://www.basketball-reference.com/leagues/NBA_2018_games.html
# https://www.basketball-reference.com/teams/DEN/2021_games.html
# https://www.basketball-reference.com/boxscores/202012230POR.html

    

Schedule 1.053 seconds url: https://www.basketball-reference.com//leagues/NBA_2018_games.html
{'link': 'https://www.basketball-reference.com//boxscores/201710170CLE.html', 'archivo': 'sessons/2018.txt', 'fecha': '710170CLE'}
{'link': 'https://www.basketball-reference.com//boxscores/201710170GSW.html', 'archivo': 'sessons/2018.txt', 'fecha': '710170GSW'}
{'link': 'https://www.basketball-reference.com//boxscores/201710180DET.html', 'archivo': 'sessons/2018.txt', 'fecha': '710180DET'}
{'link': 'https://www.basketball-reference.com//boxscores/201710180IND.html', 'archivo': 'sessons/2018.txt', 'fecha': '710180IND'}
{'link': 'https://www.basketball-reference.com//boxscores/201710180ORL.html', 'archivo': 'sessons/2018.txt', 'fecha': '710180ORL'}
{'link': 'https://www.basketball-reference.com//boxscores/201710180WAS.html', 'archivo': 'sessons/2018.txt', 'fecha': '710180WAS'}
{'link': 'https://www.basketball-reference.com//boxscores/201710180BOS.html', 'archivo': 'sessons/2018.txt', 'fecha': '7

In [37]:
!pip install --upgrade pip